In [1]:
import skimage
import os
from skimage import io
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.color import rgb2hsv
import re
import numpy as np
import seaborn as sns
## for the stats class
import scipy.signal as sg
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.decomposition import SparsePCA
import pywt
from scipy import stats
import timeit 
#import optshrink as opt # package we create
import numpy as np
# import scipy.io as sio
import h5py
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import seaborn as sns
import pandas as pd
from datetime import datetime
import pytz
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm
from skimage import io
from patchify import patchify, unpatchify
%matplotlib inline



import collections
# from itertools import chain
# import urllib.request as request
# import pickle 

import numpy as np

import scipy.signal as signal
import scipy.special as special
import scipy.optimize as optimize

import matplotlib.pyplot as plt

import skimage.io
import skimage.transform

# import cv2

# from libsvm import svmutil
from sklearn.model_selection import train_test_split



In [13]:
fog_aware_pivot = pd.read_csv('fog_aware_stats_pivot.csv')
fog_aware_pivot['Label_train'] = 1*(fog_aware_pivot['Label']=='foggy')

In [14]:
fog_aware_pivot[['Label_train', 'Label']].value_counts()

Label_train  Label    
0            not_foggy    6851
1            foggy        1776
dtype: int64

In [28]:
from sklearn import preprocessing
X = np.array(fog_aware_pivot[['mscn_var_0', 'mscn_var_1',
       'mscn_var_2', 'mscn_var_3', 'vertical_var_0', 'vertical_var_1',
       'vertical_var_2', 'vertical_var_3', 'sharpness_0', 'sharpness_1',
       'sharpness_2', 'sharpness_3', 'coef_or_var_sharpness_0',
       'coef_or_var_sharpness_1', 'coef_or_var_sharpness_2',
       'coef_or_var_sharpness_3', 'entropy_0',
       'entropy_1', 'entropy_2', 'entropy_3', 
       'color_sat_0', 'color_sat_1', 'color_sat_2', 'color_sat_3', 'CF_0',
       'CF_1', 'CF_2', 'CF_3']].dropna()
            )
####scale to mean and std

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

y = np.array(fog_aware_pivot['Label_train'])
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=2)     
# next(skf.split(X, y))
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.20, random_state=42, stratify = y)

n_classes = len(np.unique(y_train))


In [29]:
##### logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_train_pred = clf.predict(X_train)
train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel()) 
print( 'Train accuracy: %.2f' % train_accuracy)

y_test_pred = clf.predict(X_test)
test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())
print('Test accuracy: %.2f' % test_accuracy)


Train accuracy: 0.92
Test accuracy: 0.92


/Users/sonra/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
##### support vector machine
from sklearn import svm
regr = svm.SVR()
regr.fit(X_train, y_train)

threshold = 0.5

y_train_pred = regr.predict(X_train)
# Apply the logic to convert probabilities to binary values
y_train_pred_binary = (y_train_pred > threshold).astype(int)
train_accuracy = np.mean(y_train_pred_binary.ravel() == y_train.ravel()) 
print( 'Train accuracy: %.2f' % train_accuracy)

y_test_pred = regr.predict(X_test)
y_test_pred_binary = (y_test_pred > threshold).astype(int)
test_accuracy = np.mean(y_test_pred_binary.ravel() == y_test.ravel())
print('Test accuracy: %.2f' % test_accuracy)


Train accuracy: 0.94
Test accuracy: 0.92


In [33]:
y_train

array([0, 0, 1, ..., 1, 0, 0])

In [ ]:
#### random forest

In [20]:
#### GMM

gmm = GaussianMixture(n_components=n_classes
                    # ,covariance_type='diag'
                      , init_params='random_from_data')
gmm.fit(X_train)
y_train_pred = gmm.predict(X_train)
train_accuracy = np.mean(y_train_pred == y_train) 
print( 'Train accuracy: %.2f' % train_accuracy)

y_test_pred = gmm.predict(X_test)
test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())
print('Test accuracy: %.2f' % test_accuracy)


Train accuracy: 0.49
Test accuracy: 0.49
